In [ ]:
#basics
import numpy as np
import pandas as pd
import csv
import os
import sys
import sklearn
import tensorflow as tf
from tensorflow import keras
import math

#modeling
from keras import backend as K
from keras.models import Model
from keras.layers import *
import tensorflow_addons as tfa
from sklearn.model_selection import train_test_split
#import transformers

#image preprocessing
from keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from PIL import Image
import cv2
from keras.optimizers import Adam
from keras.metrics import categorical_accuracy

#visualization
import matplotlib.pyplot as plt
#import seaborn as sns

#warnings
import warnings
warnings.filterwarnings('ignore')

K.clear_session()
np.random.seed(2021); tf.random.set_seed(2021)


In [ ]:
train = pd.read_csv('../input/shopee-product-matching/train.csv')

label_dict = train.groupby('label_group').posting_id.agg('unique').to_dict()
new_label_dict = {}
label_counts = 1
for k, v in label_dict.items():
    if len(v) <= 3:
        new_label_dict[k] = 0
    else:
        new_label_dict[k] = label_counts
        label_counts += 1
num_class = label_counts


In [ ]:
train_jpg_directory = '../input/shopee-product-matching/train_images'

train_image_path = []
for img in train.image:
    train_image_path.append(os.path.join(train_jpg_directory, img))
train['img_path'] = train_image_path

N_tot = len(train)
all_ind = np.arange(N_tot)


In [ ]:
LIMIT = 15
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],
        [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=1024*LIMIT)])
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    #print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    print(e)

Rough idea is that, we can apply Additive Angular Margin Loss proposed in "ArcFace: Additive Angular Margin Loss for Deep Face Recognition" to firstly do the classification. It is by finely tuning a pretrained model over the training dataset. After that, we extract CNN features from the deep model and then compute the similarity score between images, with which the ranking list is obtained. A threshold $\theta$ is used to cut the ranking list.

Below is the ArcFace layer for classification

In [ ]:
from keras.metrics import categorical_accuracy
from keras import regularizers

import math

from keras.metrics import categorical_accuracy

import tensorflow as tf
import math as m


class ArcFace(Layer):
    '''Custom Keras layer implementing ArcFace including:
    1. Generation of embeddings
    2. Loss function
    3. Accuracy function
    '''

    def __init__(self, output_dim, class_num, margin=0.5, scale=64., **kwargs):
        self.output_dim = output_dim
        self.class_num = class_num
        self.margin = margin
        self.s = scale

        self.cos_m = tf.math.cos(margin)
        self.sin_m = tf.math.sin(margin)
        self.mm = self.sin_m * margin
        self.threshold = tf.math.cos(tf.constant(m.pi) - margin)
        super(ArcFace, self).__init__(**kwargs)


    def build(self, input_shape):
        # Create a trainable weight variable for this layer.
        self.kernel = self.add_weight(name='kernel', 
                                      shape=(input_shape[1], self.class_num),
                                      initializer='glorot_normal',
                                      trainable=True)
        super(ArcFace, self).build(input_shape)  # Be sure to call this at the end


    def call(self, x):
        embeddings = tf.nn.l2_normalize(x, axis=1, name='normed_embeddings')
        weights = tf.nn.l2_normalize(self.kernel, axis=0, name='normed_weights')
        cos_t = tf.matmul(embeddings, weights, name='cos_t')
        return cos_t


    def get_logits(self, labels, y_pred):
        cos_t = y_pred
        cos_t2 = tf.square(cos_t, name='cos_2')
        sin_t2 = tf.subtract(1., cos_t2, name='sin_2')
        sin_t = tf.sqrt(sin_t2, name='sin_t')
        cos_mt = self.s * tf.subtract(tf.multiply(cos_t, self.cos_m), tf.multiply(sin_t, self.sin_m), name='cos_mt')
        cond_v = cos_t - self.threshold
        cond = tf.cast(tf.nn.relu(cond_v, name='if_else'), dtype=tf.bool)
        keep_val = self.s*(cos_t - self.mm)
        cos_mt_temp = tf.where(cond, cos_mt, keep_val)
        mask = tf.one_hot(labels, depth=self.class_num, name='one_hot_mask')
        inv_mask = tf.subtract(1., mask, name='inverse_mask')
        s_cos_t = tf.multiply(self.s, cos_t, name='scalar_cos_t')
        output = tf.add(tf.multiply(s_cos_t, inv_mask), tf.multiply(cos_mt_temp, mask), name='arcface_logits')
        return output


    def loss(self, y_true, y_pred):
        labels = K.argmax(y_true, axis=-1)
        logits = self.get_logits(labels, y_pred)
        loss = tf.compat.v1.losses.sparse_softmax_cross_entropy(labels=labels, logits=logits)
        return loss


    def accuracy(self, y_true, y_pred):
        labels = K.argmax(y_true, axis=-1)
        logits = self.get_logits(labels, y_pred)
        accuracy = categorical_accuracy(y_true=labels, y_pred=logits)
        return accuracy
    

    def compute_output_shape(self, input_shape):
        return (input_shape[0], self.output_dim)


from tensorflow.keras.applications import EfficientNetB4
model = EfficientNetB4(include_top=False, weights='../input/tfkerasefficientnetimagenetnotop/efficientnetb4_notop.h5', pooling='avg', input_shape=None)

af_layer = ArcFace(output_dim=num_class, class_num=num_class, margin=0.5, scale=30.)
#Y_input =  Input(shape=(num_class,))

x = Dropout(rate = 0.2, name = "added_dropout")(model.output)
x = Dense(units = 512, name = "added_fc_layer")(x)
x = BatchNormalization(name = "added_bn_layer")(x)

af_output = af_layer(x)

model = Model(inputs=model.input, outputs=af_output)

We use the ResNet as backbone for feature learning.

In [ ]:
import skimage.io
import skimage.transform
from sklearn import *

#AUTO = tf.data.experimental.AUTOTUNE
IMAGE_SIZE = [256, 256]

from tensorflow.keras.applications import EfficientNetB4
model = EfficientNetB4(include_top=False, weights=None, pooling='avg', input_shape=None)
model.load_weights('../input/finetune2/my_model_weights_fine_tune4.h5', by_name=True)

def create_batch(train, batch_index):
    x_inputs = np.zeros((len(batch_index), 256, 256, 3))
    x_labels = np.zeros((len(batch_index), num_class))
    for i, iindex in enumerate(batch_index):
        x_anchor = train.iloc[iindex].img_path
        x_anchor = tf.io.read_file(x_anchor)
        x_anchor = tf.image.decode_jpeg(x_anchor, channels=3)
        x_anchor = tf.image.resize(x_anchor, IMAGE_SIZE)

        x_inputs[i] = x_anchor
        x_labels[i, new_label_dict[train.iloc[iindex].label_group]] = 1.0
    

    return x_inputs, x_labels


train_set = np.arange(N_tot)
train_index = train_set[2*10: 3*10]
x_data, y_data = create_batch(train, train_index)
print(y_data)

y_pred = model.predict(x_data)
print(y_pred)


In [ ]:
# lr_schedule = keras.optimizers.schedules.ExponentialDecay(
#     initial_learning_rate=1e-3,
#     decay_steps=15000,
#     decay_rate=0.8)

# op = Adam(learning_rate=lr_schedule, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
# #model.compile(optimizer=op, loss=arcf.loss, metrics=[arcf.accuracy])

# train_set = np.arange(N_tot)
# BATCH_SIZE = 64
# EPOCH = 10
# train_size = len(train_set)
# ITER = int(train_size/BATCH_SIZE)

# loss_fn = af_layer.loss
# train_acc_metric = af_layer.accuracy


# for iepoch in range(EPOCH):
#     np.random.shuffle(train_set)
#     print("Training epoch %d --------"%(iepoch))
#     for it in range(ITER):
#         train_index = train_set[it*BATCH_SIZE: (it+1)*BATCH_SIZE]
#         x_data, y_data = create_batch(train, train_index)
#         #print (np.argmax(y_data, axis = 1))
#         with tf.GradientTape() as tape:
#             y_pred = model(x_data, training=True)
#             loss_value = loss_fn(y_data, y_pred)
#         grads = tape.gradient(loss_value, model.trainable_weights)
#         op.apply_gradients(zip(grads, model.trainable_weights))
        
#         if it % 20 == 0:
#             batch_acc =  train_acc_metric(y_data, y_pred)
#             print("Seen so far: %s samples, Training loss at step %d: %.4f, Acc by far %.4f"% (((it + 1) * BATCH_SIZE), it, float(loss_value), np.mean(batch_acc)))
    
#     #print("Training acc over epoch: %.4f" % (float(train_acc),))
#     if (iepoch + 1) % 2 == 0:
#         mwp = "my_model_weights_fine_tune" + str(iepoch + 1)+ ".h5"
#         model.save_weights(mwp)
